# Big Data Project
## Outcome: Divergence

In this notebook we will outcome 'Divergence'

'Divergence' will be used to detect matches with a big difference between bet houses quotas.


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, DataType, DecimalType
from pyspark.sql.functions import udf
import pyspark

spark = SparkSession \
    .builder \
    .appName("Bets Exploring") \
    .getOrCreate()
    
sc = spark.sparkContext

sc

<SparkContext master=local[*] appName=Bets Exploring>

In [2]:
pathori = "../Data/Raw/Main/"
pathtemp = "../Data/Interim/"
pathdest = "../Data/Processed/"
filetemp = "main_competitions.csv"
filedest = "main_competitions_recent.csv"
bethouses = ['B365', 'BS', 'BW', 'GB', 'IW', 'LB', 'SB', 'SJ', 'VC', 'WH']
quotatypes = ['H','D','A']

In [3]:
# Open dataset
df = spark.read.csv(path = pathdest + filedest, header = True, inferSchema = True)
df = df.withColumn("Date", df["Date"].cast("date"))

In [4]:
print ("Size:   ", df.count())
print ("Columns:", df.columns)
df.printSchema()

Size:    11374
Columns: ['Country', 'Competition', 'Season', 'Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'VCH', 'VCD', 'VCA', 'WHH', 'WHD', 'WHA']
root
 |-- Country: string (nullable = true)
 |-- Competition: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- Div: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- HomeTeam: string (nullable = true)
 |-- AwayTeam: string (nullable = true)
 |-- FTR: string (nullable = true)
 |-- B365H: double (nullable = true)
 |-- B365D: double (nullable = true)
 |-- B365A: double (nullable = true)
 |-- BWH: double (nullable = true)
 |-- BWD: double (nullable = true)
 |-- BWA: double (nullable = true)
 |-- IWH: double (nullable = true)
 |-- IWD: double (nullable = true)
 |-- IWA: double (nullable = true)
 |-- LBH: double (nullable = true)
 |-- LBD: double (nullable = true)
 |-- LBA: double (nullable = true)
 |-- VCH: double (nullable

In [5]:
df.show(5)

+-------+-------------+---------+---+----------+---------+----------------+---+-----+-----+-----+----+---+----+----+---+---+----+---+----+----+----+----+----+----+----+
|Country|  Competition|   Season|Div|      Date| HomeTeam|        AwayTeam|FTR|B365H|B365D|B365A| BWH|BWD| BWA| IWH|IWD|IWA| LBH|LBD| LBA| VCH| VCD| VCA| WHH| WHD| WHA|
+-------+-------------+---------+---+----------+---------+----------------+---+-----+-----+-----+----+---+----+----+---+---+----+---+----+----+----+----+----+----+----+
|Belgium|JupilerLeague|2017-2018| B1|2017-07-28|  Antwerp|      Anderlecht|  D| 5.75|  3.8|  1.6| 5.0|4.0|1.67| 4.7|3.6|1.7| 5.2|3.7|1.61| 5.5| 3.9|1.62|4.75|3.75|1.67|
|Belgium|JupilerLeague|2017-2018| B1|2017-07-29|Charleroi|        Kortrijk|  H| 1.62| 3.75|  5.5|1.67|3.7| 5.5| 1.6|3.7|5.4|1.61|3.6| 5.2|1.65| 3.8|5.25|1.65|3.75| 4.8|
|Belgium|JupilerLeague|2017-2018| B1|2017-07-29|    Eupen|         Waregem|  A|  2.6|  3.4|  2.6| 2.7|3.4| 2.6|2.65|3.3|2.5|2.55|3.3| 2.5|2.63| 3.4| 2.6| 2

In [6]:
df.count()

11374

In [7]:
df.describe('Country','Competition','Season','Div','Date','HomeTeam','AwayTeam','FTR').show()

+-------+-------+------------+---------+-----+--------+--------+-----+
|summary|Country| Competition|   Season|  Div|HomeTeam|AwayTeam|  FTR|
+-------+-------+------------+---------+-----+--------+--------+-----+
|  count|  11374|       11374|    11374|11374|   11374|   11374|11374|
|   mean|   null|        null|     null| null|    null|    null| null|
| stddev|   null|        null|     null| null|    null|    null| null|
|    min|Belgium|Championship|2017-2018|   B1|     AEK|     AEK|    A|
|    max| Turkey|      SerieB|2018-2019|   T1|  Zwolle|  Zwolle|    H|
+-------+-------+------------+---------+-----+--------+--------+-----+



In [9]:
# df.describe('WHH','WHD','WHA','SBH','SBD','SBA','IWH','IWD','IWA','GBH','GBD','GBA').show()

In [10]:
dfok = df.dropna(how='any')
print(dfok.count())
dfok.show(5)

7668
+-------+-------------+---------+---+----------+---------+----------------+---+-----+-----+-----+----+---+----+----+---+---+----+---+----+----+----+----+----+----+----+
|Country|  Competition|   Season|Div|      Date| HomeTeam|        AwayTeam|FTR|B365H|B365D|B365A| BWH|BWD| BWA| IWH|IWD|IWA| LBH|LBD| LBA| VCH| VCD| VCA| WHH| WHD| WHA|
+-------+-------------+---------+---+----------+---------+----------------+---+-----+-----+-----+----+---+----+----+---+---+----+---+----+----+----+----+----+----+----+
|Belgium|JupilerLeague|2017-2018| B1|2017-07-28|  Antwerp|      Anderlecht|  D| 5.75|  3.8|  1.6| 5.0|4.0|1.67| 4.7|3.6|1.7| 5.2|3.7|1.61| 5.5| 3.9|1.62|4.75|3.75|1.67|
|Belgium|JupilerLeague|2017-2018| B1|2017-07-29|Charleroi|        Kortrijk|  H| 1.62| 3.75|  5.5|1.67|3.7| 5.5| 1.6|3.7|5.4|1.61|3.6| 5.2|1.65| 3.8|5.25|1.65|3.75| 4.8|
|Belgium|JupilerLeague|2017-2018| B1|2017-07-29|    Eupen|         Waregem|  A|  2.6|  3.4|  2.6| 2.7|3.4| 2.6|2.65|3.3|2.5|2.55|3.3| 2.5|2.63| 3.4| 2

In [13]:
# Bet Houses selection depending on dataset simulation
print (bethouses)
for bethouse in bethouses:
    col = bethouse + 'H'
    if not col in dfok.columns:
        bethouses.remove(bethouse)
        
print (bethouses)

['B365', 'BW', 'IW', 'LB', 'SJ', 'VC', 'WH']
['B365', 'BW', 'IW', 'LB', 'VC', 'WH']


In [14]:
# Divergence calculation simulation
match = dfok.take(1)[0]

resultList = ['H','D','A']
divergence = 0
for result in resultList:
    print ()
    min = 1000
    max = 0
    sum = 0
    count = 0
    for bethouse in bethouses:
        col = bethouse + result
        val = match[col]
        if not val is None:
            print (col, ':', val, end='  ')
            sum = sum + val
            count = count + 1
            if val > max:
                max = val
            if val < min:
                min = val
    mean = sum/count
    diverMax = (max / mean) - 1
    if diverMax > divergence:
        divergence = diverMax
    diverMin = abs((min / mean) - 1)
    if diverMin > divergence:
        divergence = diverMin
    
    print ('\nCount:', count, end='  ')
    print ('Sum:', sum, end='  ')
    print ('Max:', max, end='  ')
    print ('Min:', min, end='  ')
    print ('Mean:', mean)
    print ('Diver Max:  ', diverMax, end='  ')
    print ('Diver Min:  ', diverMin, end='  ')
    print ('Divergence: ', divergence)
       


B365H : 5.75  BWH : 5.0  IWH : 4.7  LBH : 5.2  VCH : 5.5  WHH : 4.75  
Count: 6  Sum: 30.9  Max: 5.75  Min: 4.7  Mean: 5.1499999999999995
Diver Max:   0.11650485436893221  Diver Min:   0.08737864077669888  Divergence:  0.11650485436893221

B365D : 3.8  BWD : 4.0  IWD : 3.6  LBD : 3.7  VCD : 3.9  WHD : 3.75  
Count: 6  Sum: 22.75  Max: 4.0  Min: 3.6  Mean: 3.7916666666666665
Diver Max:   0.05494505494505497  Diver Min:   0.05054945054945048  Divergence:  0.11650485436893221

B365A : 1.6  BWA : 1.67  IWA : 1.7  LBA : 1.61  VCA : 1.62  WHA : 1.67  
Count: 6  Sum: 9.87  Max: 1.7  Min: 1.6  Mean: 1.6449999999999998
Diver Max:   0.033434650455927084  Diver Min:   0.027355623100303816  Divergence:  0.11650485436893221


In [15]:
def filterBetHouses(df, bethouses):
    for bethouse in bethouses:
        col = bethouse + 'H'
        if not col in df.columns:
            bethouses.remove(bethouse)

    return bethouses

In [16]:
def calcDivergence(match, bethouses) :
    resultList = ['H','D','A']
    divergence = 0
    for result in resultList:
        min = 1000
        max = 0
        sum = 0
        count = 0
        for bethouse in bethouses:
            col = bethouse + result
            val = match[col]
            if not val is None:
                sum = sum + val
                count = count + 1
                if val > max:
                    max = val
                if val < min:
                    min = val
        mean = sum/count
        diverMax = (max / mean) - 1
        if diverMax > divergence:
            divergence = diverMax
        diverMin = abs((min / mean) - 1)
        if diverMin > divergence:
            divergence = diverMin
            
    return divergence

In [19]:
# Map divergence
bethouses = filterBetHouses(dfok, bethouses)
rdddiver = dfok.rdd.map(lambda match: (match, calcDivergence(match, bethouses)))
print ("Bet Housess:", bethouses)
rdddiver.take(5)

Bet Housess: ['B365', 'BW', 'IW', 'LB', 'VC', 'WH']


[(Row(Country='Belgium', Competition='JupilerLeague', Season='2017-2018', Div='B1', Date=datetime.date(2017, 7, 28), HomeTeam='Antwerp', AwayTeam='Anderlecht', FTR='D', B365H=5.75, B365D=3.8, B365A=1.6, BWH=5.0, BWD=4.0, BWA=1.67, IWH=4.7, IWD=3.6, IWA=1.7, LBH=5.2, LBD=3.7, LBA=1.61, VCH=5.5, VCD=3.9, VCA=1.62, WHH=4.75, WHD=3.75, WHA=1.67),
  0.11650485436893221),
 (Row(Country='Belgium', Competition='JupilerLeague', Season='2017-2018', Div='B1', Date=datetime.date(2017, 7, 29), HomeTeam='Charleroi', AwayTeam='Kortrijk', FTR='H', B365H=1.62, B365D=3.75, B365A=5.5, BWH=1.67, BWD=3.7, BWA=5.5, IWH=1.6, IWD=3.7, IWA=5.4, LBH=1.61, LBD=3.6, LBA=5.2, VCH=1.65, VCD=3.8, VCA=5.25, WHH=1.65, WHD=3.75, WHA=4.8),
  0.09004739336492884),
 (Row(Country='Belgium', Competition='JupilerLeague', Season='2017-2018', Div='B1', Date=datetime.date(2017, 7, 29), HomeTeam='Eupen', AwayTeam='Waregem', FTR='A', B365H=2.6, B365D=3.4, B365A=2.6, BWH=2.7, BWD=3.4, BWA=2.6, IWH=2.65, IWD=3.3, IWA=2.5, LBH=2.55,

In [20]:
# Help me: Trying to addcolumn in DataFrame
# Option 1
dfdiver = dfok.withColumn('Divergence', calcDivergence(dfok, bethouses))
dfdiver.show(5)

ValueError: Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.

In [21]:
# Option 2
to_diver = udf(lambda match: calcDivergence(match, bethouses, DecimalType()))
dfdiver = dfok.withColumn('Divergence', to_diver(dfok))
dfdiver.show(5)

TypeError: Invalid argument, not a string or column: DataFrame[Country: string, Competition: string, Season: string, Div: string, Date: date, HomeTeam: string, AwayTeam: string, FTR: string, B365H: double, B365D: double, B365A: double, BWH: double, BWD: double, BWA: double, IWH: double, IWD: double, IWA: double, LBH: double, LBD: double, LBA: double, VCH: double, VCD: double, VCA: double, WHH: double, WHD: double, WHA: double] of type <class 'pyspark.sql.dataframe.DataFrame'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

In [22]:
# Top N highest divergence
rdddiver.top(5, key=lambda match: match[1])

[(Row(Country='France', Competition='LeChampionnat', Season='2017-2018', Div='F1', Date=datetime.date(2018, 4, 29), HomeTeam='Paris SG', AwayTeam='Guingamp', FTR='D', B365H=1.08, B365D=11.0, B365A=26.0, BWH=1.07, BWD=12.0, BWA=29.0, IWH=1.1, IWD=8.0, IWA=20.0, LBH=1.08, LBD=12.0, LBA=34.0, VCH=1.06, VCD=13.0, VCA=46.0, WHH=1.11, WHD=8.5, WHA=21.0),
  0.5681818181818183),
 (Row(Country='France', Competition='LeChampionnat', Season='2017-2018', Div='F1', Date=datetime.date(2017, 12, 20), HomeTeam='Paris SG', AwayTeam='Caen', FTR='H', B365H=1.07, B365D=12.0, B365A=29.0, BWH=1.08, BWD=11.5, BWA=26.0, IWH=1.1, IWD=8.0, IWA=20.0, LBH=1.07, LBD=12.0, LBA=41.0, VCH=1.05, VCD=13.0, VCA=51.0, WHH=1.06, WHD=12.0, WHA=29.0),
  0.5612244897959184),
 (Row(Country='Portugal', Competition='Liga1', Season='2017-2018', Div='P1', Date=datetime.date(2018, 4, 28), HomeTeam='Benfica', AwayTeam='Tondela', FTR='A', B365H=1.12, B365D=8.0, B365A=19.0, BWH=1.08, BWD=9.5, BWA=23.0, IWH=1.08, IWD=10.0, IWA=25.0, L

In [23]:
rdddiver.top(5, key=lambda match: -match[1])

[(Row(Country='Spain', Competition='Primera', Season='2017-2018', Div='SP1', Date=datetime.date(2018, 1, 4), HomeTeam='Leganes', AwayTeam='Valencia', FTR='A', B365H=3.75, B365D=3.39, B365A=2.0, BWH=3.8, BWD=3.25, BWA=2.0, IWH=3.8, IWD=3.4, IWA=2.0, LBH=3.75, LBD=3.25, LBA=2.05, VCH=3.75, VCD=3.4, VCA=2.05, WHH=3.75, WHD=3.25, WHA=2.05),
  0.02306920762286846),
 (Row(Country='England', Competition='Premier', Season='2017-2018', Div='E0', Date=datetime.date(2018, 10, 3), HomeTeam='Newcastle', AwayTeam='Southampton', FTR='H', B365H=2.79, B365D=3.2, B365A=2.79, BWH=2.7, BWD=3.1, BWA=2.8, IWH=2.7, IWD=3.2, IWA=2.7, LBH=2.7, LBD=3.1, LBA=2.7, VCH=2.8, VCD=3.2, VCA=2.8, WHH=2.8, WHD=3.1, WHA=2.8),
  0.023508137432188048),
 (Row(Country='Germany', Competition='Liga2', Season='2017-2018', Div='D2', Date=datetime.date(2017, 3, 12), HomeTeam='Bochum', AwayTeam='Union Berlin', FTR='H', B365H=2.62, B365D=3.39, B365A=2.54, BWH=2.7, BWD=3.3, BWA=2.6, IWH=2.65, IWD=3.3, IWA=2.5, LBH=2.6, LBD=3.3, LBA=